# Transfer Learning with TensorFlow: Fine-Tuning

In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir

## Get data

In [3]:
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [4]:
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [5]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [6]:
train_data_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [7]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[1.19751595e+02 1.07751595e+02 1.07751595e+02]
   [1.21745537e+02 1.09745537e+02 1.09745537e+02]
   [1.23039223e+02 1.11039223e+02 1.11039223e+02]
   ...
   [7.90090561e+01 6.30090599e+01 6.37947960e+01]
   [9.00227356e+01 7.50227356e+01 7.80227356e+01]
   [7.74074249e+01 6.54074249e+01 6.94074249e+01]]

  [[1.19937180e+02 1.07937180e+02 1.07937180e+02]
   [1.20967796e+02 1.08967796e+02 1.08967796e+02]
   [1.23096619e+02 1.11096619e+02 1.11096619e+02]
   ...
   [9.75188370e+01 8.15188370e+01 8.23045731e+01]
   [8.44631042e+01 6.94631042e+01 7.24631042e+01]
   [8.47994003e+01 7.27994003e+01 7.67994003e+01]]

  [[1.20084824e+02 1.08084824e+02 1.08084824e+02]
   [1.20084824e+02 1.08084824e+02 1.08084824e+02]
   [1.23196114e+02 1.11196114e+02 1.11196114e+02]
   ...
   [8.48886642e+01 6.88886642e+01 6.96744003e+01]
   [9.36168823e+01 7.86168823e+01 8.16168823e+01]
   [7.25082703e+01 6.05082703e+01 6.45082703e+01]]

  ...

  [[7.52982254e+01 6.92982254e+01 7.92982254e+01]
   [7

## Model 0: Building a transfer learning model using the Keras Functional API

In [9]:
# 1. Create the base model with tf.keras.applications
base_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
# 2. Freeze the model
base_model.trainable = False

# 3. Create inputs into the model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. If using a model like ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool of the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

history_0 = model_0.fit(
    train_data_10_percent,
    steps_per_epoch=len(train_data_10_percent),
    epochs=20,
    verbose=True,
    callbacks=[
        create_tensorboard_callback("05_tensorboard_logs", "10_percent_feature_extraction")
    ],
    validation_data=test_data,
    validation_steps=int(0.25 * len(test_data))
)


16711680/16705208 [==============================] - 1s 0us/step
Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)
Saving TensorBoard log files to: 05_tensorboard_logs/10_percent_feature_extraction/20210813-103845


C:\ProgramData\Anaconda3\envs\general_env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/20
24/24 [==============================] - 18s 263ms/step - loss: 1.8650 - accuracy: 0.4107 - val_loss: 1.3090 - val_accuracy: 0.7516
Epoch 2/20
24/24 [==============================] - 3s 131ms/step - loss: 1.1330 - accuracy: 0.7573 - val_loss: 0.9022 - val_accuracy: 0.8059
Epoch 3/20
24/24 [==============================] - 3s 130ms/step - loss: 0.8337 - accuracy: 0.8067 - val_loss: 0.7327 - val_accuracy: 0.8191
Epoch 4/20
24/24 [==============================] - 3s 127ms/step - loss: 0.6728 - accuracy: 0.8480 - val_loss: 0.6447 - val_accuracy: 0.8454
Epoch 5/20
24/24 [==============================] - 3s 129ms/step - loss: 0.5798 - accuracy: 0.8693 - val_loss: 0.5955 - val_accuracy: 0.8487
Epoch 6/20
24/24 [==============================] - 3s 130ms/step - loss: 0.5070 - accuracy: 0.8827 - val_loss: 0.5650 - val_accuracy: 0.8421
Epoch 7/20
24/24 [==============================] - 3s 128ms/step - loss: 0.4642 - accuracy: 0.8987 - val_loss: 0.5111 - val_accuracy: 0.8569
Epoch

In [10]:
model_0.evaluate(test_data)

79/79 [==============================] - 7s 84ms/step - loss: 0.4359 - accuracy: 0.8568


[0.4358503818511963, 0.8568000197410583]

In [11]:
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name)

0 input_1
1 rescaling
2 normalization
3 stem_conv_pad
4 stem_conv
5 stem_bn
6 stem_activation
7 block1a_dwconv
8 block1a_bn
9 block1a_activation
10 block1a_se_squeeze
11 block1a_se_reshape
12 block1a_se_reduce
13 block1a_se_expand
14 block1a_se_excite
15 block1a_project_conv
16 block1a_project_bn
17 block2a_expand_conv
18 block2a_expand_bn
19 block2a_expand_activation
20 block2a_dwconv_pad
21 block2a_dwconv
22 block2a_bn
23 block2a_activation
24 block2a_se_squeeze
25 block2a_se_reshape
26 block2a_se_reduce
27 block2a_se_expand
28 block2a_se_excite
29 block2a_project_conv
30 block2a_project_bn
31 block2b_expand_conv
32 block2b_expand_bn
33 block2b_expand_activation
34 block2b_dwconv
35 block2b_bn
36 block2b_activation
37 block2b_se_squeeze
38 block2b_se_reshape
39 block2b_se_reduce
40 block2b_se_expand
41 block2b_se_excite
42 block2b_project_conv
43 block2b_project_bn
44 block2b_drop
45 block2b_add
46 block3a_expand_conv
47 block3a_expand_bn
48 block3a_expand_activation
49 block3a_dwcon

In [12]:
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, None, None, 3 7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, None, None, 3 0           normalization[0][0]              
_____________________________________________________________________________________

In [ ]:
model_0.summary()